In [3]:
print("hello world")

hello world


In [2]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

In [5]:
!wget https://www.crcv.ucf.edu/data/UCF101/UCF101.rar -y

# unrar e UCF101.rar Videos/

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# open the .txt file which have names of training videos
f = open("trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()

FileNotFoundError: [Errno 2] No such file or directory: 'trainlist01.txt'